In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
#%matplotlib tk

In [534]:
result = pd.read_csv("/home/lin/git/summerProject/S0100_numerical_goodKs")

In [535]:
result[['time', 'iteration', 'pv', 'jac', 'lin_sys', 'e0', 'e*', 'Je', 'Dp', 'count','count2', 'count7']].describe()

,time,iteration,pv,jac,lin_sys,e0,e*,Je,Dp,count,count2,count7
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1.000000e+03,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.523195,33.835000,54.123000,3.861000,33.835000,3.331165e+09,2.178583e-07,2.191805e-04,0.011609,1.066000,0.064000,0.002000
std,0.669642,30.720507,45.578715,2.989082,30.720507,1.033005e+11,3.653453e-06,4.119327e-03,0.363885,0.660364,0.632696,0.044699
min,0.115288,8.000000,14.000000,1.000000,8.000000,4.289220e-04,2.047878e-16,9.815008e-11,0.000000,1.000000,0.000000,0.000000
25%,0.276738,20.000000,31.000000,2.000000,20.000000,5.676479e-02,3.499831e-12,5.842737e-06,0.000003,1.000000,0.000000,0.000000
50%,0.380937,26.000000,42.000000,3.000000,26.000000,1.560605e-01,1.823104e-11,1.304264e-05,0.000014,1.000000,0.000000,0.000000
75%,0.557786,38.250000,60.000000,4.000000,38.250000,6.169299e-01,5.387359e-11,3.318051e-05,0.000049,1.000000,0.000000,0.000000
max,12.154593,300.000000,456.000000,31.000000,300.000000,3.266230e+12,1.033219e-04,1.281911e-01,11.507144,19.000000,17.000000,1.000000


In [536]:
result.columns

Index(['Ik', 'Ivbar', 'Iv0', 'Irho', 'Isigma', 'Ok', 'Rk', 'Ovbar', 'Rvbar',
       'Ov0', 'Rv0', 'Orho', 'Rrho', 'Osigma', 'Rsigma', 'stopped_by', 'time',
       'iteration', 'pv', 'jac', 'lin_sys', 'e0', 'e*', 'Je', 'Dp', 'count',
       'count2', 'count7'],
      dtype='object')

In [537]:
result['stopped_by'].unique() #3, itmax, 1 small gradient, 5 increase mu

array([6, 3, 1])

In [538]:
solved = result[result['stopped_by'] == 6]
solved.count().unique()

array([991])

In [539]:
solved.count().unique()/result.count().unique()

array([0.991])

In [540]:
solved_first_time = solved[solved['count'] == 1]
solved_first_time.count().unique()

array([960])

In [541]:
DP = result[result['stopped_by'] == 2]
DP.count().unique()

array([0])

In [542]:
DP

,Ik,Ivbar,Iv0,Irho,Isigma,Ok,Rk,Ovbar,Rvbar,Ov0,...,pv,jac,lin_sys,e0,e*,Je,Dp,count,count2,count7


In [543]:
itmax = result[result['stopped_by'] == 3]
itmax.count().unique()

array([8])

In [544]:
smalJac = result[result['stopped_by'] == 1]
smalJac.count().unique()

array([1])

In [545]:
solved['time'].describe()

count    991.000000
mean       0.495105
std        0.593314
min        0.115288
25%        0.274034
50%        0.379607
75%        0.553263
max       12.154593
Name: time, dtype: float64

In [546]:
solved_first_time['time'].describe()

count    960.000000
mean       0.442017
std        0.239869
min        0.115288
25%        0.268455
50%        0.373738
75%        0.529749
max        2.512831
Name: time, dtype: float64

In [547]:
solved_first_time['iteration'].describe()

count    960.000000
mean      31.596875
std       18.133412
min        8.000000
25%       20.000000
50%       26.000000
75%       38.000000
max      199.000000
Name: iteration, dtype: float64

In [548]:
solved['iteration'].describe()

count    991.000000
mean      31.699294
std       19.368056
min        8.000000
25%       20.000000
50%       26.000000
75%       38.000000
max      262.000000
Name: iteration, dtype: float64

In [459]:
solved.head()

,Ik,Ivbar,Iv0,Irho,Isigma,Ok,Rk,Ovbar,Rvbar,Ov0,...,pv,jac,lin_sys,e0,e*,Je,Dp,count,count2,count7
1,1.293890,0.857520,0.584743,-0.652745,0.562406,3.522433,3.522417,0.834166,0.834155,0.154045,...,18,12,17,0.041510,2.638723e-11,1.180799e-07,4.621677e-06,2,1,0
4,3.528476,0.546792,0.416693,-0.496527,0.315473,3.889896,3.889922,0.435772,0.435773,0.055080,...,22,14,21,0.195527,2.727437e-12,8.975409e-08,3.810872e-06,1,0,0
5,2.301274,0.598387,0.435573,-0.441467,0.139399,4.385548,4.385892,0.350579,0.350579,0.829674,...,25,17,24,0.021668,9.666041e-11,1.211914e-06,4.466965e-04,21,18,2
6,3.010048,0.859892,0.860849,-0.291575,0.780668,2.642079,2.642092,0.562092,0.562091,0.123273,...,27,16,26,0.477723,1.065191e-13,6.359427e-07,1.028369e-07,21,20,0
7,4.405389,0.663972,0.452743,-0.243830,0.562508,4.241974,4.241241,0.438514,0.438495,0.716152,...,30,16,29,0.017307,5.645890e-11,3.574250e-07,6.530289e-07,1,0,0


In [460]:
#'Ik', 'Ivbar', 'Iv0', 'Irho', 'Isigma', 'Ok', 'Rk', 'Ovbar', 'Rvbar',
#'Ov0', 'Rv0', 'Orho', 'Rrho', 'Osigma', 'Rsigma', 'stopped_by', 'time',
#'iteration', 'pv', 'jac', 'lin_sys', 'e0', 'e*', 'Je', 'Dp', 'count'
   

In [461]:
disIk = np.abs(solved['Ik'] - solved['Rk']).mean()
disIk

1.0271906328880622

In [462]:
disIvbar = np.abs(solved['Ivbar'] - solved['Rvbar']).mean()
disIvbar

0.2675114984299786

In [463]:
disIv0 = np.abs(solved['Iv0'] - solved['Rv0']).mean()
disIv0

0.308228138348976

In [464]:
disIsigma = np.abs(solved['Isigma'] - solved['Rsigma']).mean()
disIsigma

0.28073248352264163

In [465]:
disIrho = np.abs(solved['Irho'] - solved['Rrho']).mean()
disIrho

0.24994376861043865

In [466]:
disOk = np.abs(solved['Ok'] - solved['Rk']).mean()
disOk

0.0021064447543356897

In [467]:
disOvbar = np.abs(solved['Ovbar'] - solved['Rvbar']).mean()
disOvbar

5.6904140315868424e-05

In [468]:
disOsigma = np.abs(solved['Osigma'] - solved['Rsigma']).mean()
disOsigma

0.10412440830889465

In [469]:
disOrho = np.abs(solved['Orho'] - solved['Rrho']).mean()
disOrho

0.0959212751496506

In [470]:
disOv0 = np.abs(solved['Ov0'] - solved['Rv0']).mean()
disOv0

9.67331884692391e-06

In [471]:
solved.describe()

,Ik,Ivbar,Iv0,Irho,Isigma,Ok,Rk,Ovbar,Rvbar,Ov0,...,pv,jac,lin_sys,e0,e*,Je,Dp,count,count2,count7
count,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,...,336.000000,336.000000,336.000000,336.000000,3.360000e+02,3.360000e+02,3.360000e+02,336.000000,336.000000,336.000000
mean,2.832535,0.509868,0.440966,-0.533010,0.454663,3.024065,3.024417,0.466245,0.466221,0.447607,...,30.136905,18.883929,29.136905,0.179120,5.050264e-11,2.296013e-06,1.068530e-04,18.514881,16.910714,0.604167
std,1.287353,0.251136,0.255062,0.227341,0.254230,1.198748,1.199277,0.250532,0.250532,0.270826,...,21.472211,15.910240,21.472211,0.327359,3.025266e-11,8.608275e-06,4.891538e-04,23.455383,23.003784,2.832832
min,0.534101,0.050054,0.050363,-0.896747,0.050863,0.530870,0.530675,0.053179,0.053178,0.050948,...,7.000000,6.000000,6.000000,0.000282,1.065191e-13,7.448084e-09,6.627809e-10,1.000000,0.000000,0.000000
25%,1.736339,0.279652,0.224379,-0.736945,0.221837,2.066502,2.066337,0.251126,0.251143,0.211883,...,18.000000,12.000000,17.000000,0.026766,2.187334e-11,3.056707e-07,2.226019e-07,3.000000,2.000000,0.000000
50%,2.877741,0.512457,0.415614,-0.554655,0.453322,3.129923,3.129755,0.452656,0.452566,0.405102,...,25.000000,15.000000,24.000000,0.071790,5.212954e-11,6.399466e-07,2.001199e-06,8.000000,6.000000,0.000000
75%,3.928829,0.716604,0.653158,-0.329387,0.654901,4.033767,4.033677,0.668127,0.667981,0.692357,...,34.250000,20.000000,33.250000,0.185902,7.669068e-11,1.574637e-06,2.276655e-05,23.250000,21.250000,0.000000
max,4.997335,0.947651,0.941626,-0.106084,0.947802,4.975982,4.976532,0.949002,0.949002,0.949507,...,228.000000,220.000000,227.000000,3.207557,9.990975e-11,1.396683e-04,7.826697e-03,100.000000,98.000000,40.000000


In [472]:
solved[solved['iteration']==30]

,Ik,Ivbar,Iv0,Irho,Isigma,Ok,Rk,Ovbar,Rvbar,Ov0,...,pv,jac,lin_sys,e0,e*,Je,Dp,count,count2,count7
55,3.150125,0.592731,0.665237,-0.819938,0.086639,4.172141,4.174754,0.542865,0.542902,0.860533,...,50,30,49,0.001566,2.671610e-11,3.783577e-08,0.000003,10,9,0


In [473]:
example = solved[solved.index==567]
example

,Ik,Ivbar,Iv0,Irho,Isigma,Ok,Rk,Ovbar,Rvbar,Ov0,...,pv,jac,lin_sys,e0,e*,Je,Dp,count,count2,count7


In [34]:
example.values

array([[ 2.00310794e+00,  7.66502906e-01,  2.78598016e-01,
        -6.06551157e-01,  2.17311675e-01,  8.19289377e-01,
         8.19279086e-01,  4.52754204e-01,  4.52758708e-01,
         6.38483161e-02,  6.38483542e-02, -7.18027819e-01,
        -7.18027981e-01,  7.22171706e-01,  7.22171617e-01,
         6.00000000e+00,  4.30497000e-01,  3.00000000e+01,
         3.30000000e+01,  3.00000000e+01,  3.20000000e+01,
         1.12281824e+07,  3.97140758e-13,  2.59803814e-06,
         1.15641984e-07,  1.00000000e+00]])

In [40]:
np.abs(example['Ik'] - example['Rk'])

567    1.183829
dtype: float64

In [41]:
np.abs(example['Ivbar'] - example['Rvbar'])

567    0.313744
dtype: float64

In [42]:
np.abs(example['Iv0'] - example['Rv0'])

567    0.21475
dtype: float64

In [43]:
np.abs(example['Isigma'] - example['Rsigma'])

567    0.50486
dtype: float64

In [44]:
np.abs(example['Irho'] - example['Rrho'])

567    0.111477
dtype: float64

In [45]:
realPara = (example[['Rk', 'Rvbar', 'Rv0','Rrho', 'Rsigma']])
realPara.columns = ['k', 'vbar', 'v0', 'rho', 'sigma']
realPara.reset_index(drop=True, inplace=True)
newPara = pd.DataFrame(index=range(31), columns=realPara.columns)
for i in range(31):
    for name in realPara.columns:
        newPara.iloc[i][name] = realPara.iloc[0][name]
newPara

,k,vbar,v0,rho,sigma
0,0.819279,0.452759,0.0638484,-0.718028,0.722172
1,0.819279,0.452759,0.0638484,-0.718028,0.722172
2,0.819279,0.452759,0.0638484,-0.718028,0.722172
3,0.819279,0.452759,0.0638484,-0.718028,0.722172
4,0.819279,0.452759,0.0638484,-0.718028,0.722172
5,0.819279,0.452759,0.0638484,-0.718028,0.722172
6,0.819279,0.452759,0.0638484,-0.718028,0.722172
7,0.819279,0.452759,0.0638484,-0.718028,0.722172
8,0.819279,0.452759,0.0638484,-0.718028,0.722172
9,0.819279,0.452759,0.0638484,-0.718028,0.722172


In [47]:
paraRout = pd.read_csv("/home/lin/git/summerProject/paraRout")
paraRout

,k,vbar,v0,rho,sigma
0,2.003108,0.766503,0.278598,-0.606551,0.217312
1,1.994029,0.724699,0.189030,-0.610187,0.228518
2,1.982846,0.677274,0.140961,-0.611728,0.235546
3,1.970775,0.626121,0.111046,-0.611977,0.241220
4,1.958803,0.573471,0.089061,-0.611404,0.246896
5,1.948925,0.527419,0.067376,-0.610923,0.253178
6,1.944743,0.506576,0.042040,-0.612459,0.258198
7,1.941752,0.490938,0.022661,-0.613929,0.261750
8,1.938261,0.471839,0.007217,-0.614936,0.265345
9,1.934198,0.448370,-0.009117,-0.616059,0.269575


In [48]:
routs = np.abs(paraRout - newPara)/np.abs(newPara)

In [49]:
%matplotlib tk

In [50]:
plt.semilogy(routs['k'], 'rX-')
plt.semilogy(routs['vbar'], 'bX-')
plt.semilogy(routs['v0'], 'kX-')
plt.semilogy(routs['rho'], 'gX-')
plt.semilogy(routs['sigma'], 'mX-')
plt.legend([r'$|k_k - k^{\star}|/|k^{\star}|$', r'$|\overline{\upsilon}_k - \overline{\upsilon}^{\star}|/|\overline{\upsilon}^{\star}|$', r'$|(\upsilon_0)_k - \upsilon_0^{\star}|/|\upsilon_0^{\star}|$',r'$|\rho_k - \rho^{\star}|/|\rho^{\star}|$',r'$|\sigma_k - \sigma^{\star}|/|\sigma^{\star}|$',], fontsize=22)
plt.xlabel('Index of iteration k', fontsize=24)
plt.ylabel('Nomalized differences between parameters found and targets', fontsize=24)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid()

In [54]:
residual = np.array([11228182.46038808,807255.2860647117,76039.65750674544,10062.25865084798,1887.359525438133,351.3059154344626,49.09839337737781,10.14481475124527,3.317012356056412,1.152402586648755,0.4177379994440593,0.2055414882174497,0.1282323013866701,0.08938729119401917,0.03267351837427038,0.02753496116476249,0.02170500399809178,0.01789866338104146,0.0108093724732074,0.004463637920357317,0.001246749982923448,0.000733681009537488,0.0005813803495587621,0.0005499026977308425,0.0001869581706586826,0.0001337921316549374,1.328981933037579e-05,6.953217715155133e-08,1.436982211690292e-09,2.082707307407935e-10,3.971677980382377e-13])

In [55]:
plt.semilogy(residual, 'bX-.')
plt.legend(['Reduction path of the objective function'], fontsize=22)
plt.xlabel('Index of iteration k', fontsize=24)
plt.ylabel('Value of the objective function', fontsize=24)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid()

In [56]:
realSPX = np.array([0.07478998780360964,0.1511881858006229,0.2027213809295823,0.2410147787627178,0.2731537190898699,0.3015950796032961,0.3634066696340293,0.4259388798643055,0.03423046264597641,0.05564391048588092,0.07821984090287659,0.099843150859326,0.1197462338377685,0.1379057817951416,0.1810112295762004,0.2187702577360547,0.01335175190303648,0.02636982030737595,0.03824713876862126,0.0496783014582497,0.06059723482555364,0.07117410881843453,0.1020129531561916,0.1289858583611486,5.507656245541976e-06,0.0003417334933978206,0.001793873971183487,0.004031474194149178,0.008600842548238147,0.01362447696797399,0.03324075369386573,0.05432814885255527,-3.810543297612796e-08,2.314515999204069e-06,4.939701354645699e-05,0.0002662664419762034,0.0008058353744827404,0.001794209495300502,0.008420466884001467,0.01913449284155004])

In [57]:
realSPX.shape

(40,)

In [58]:
realVIX = np.array([0.4361447852517755,0.4361285192330442,0.4361100410116878,0.4360911262154472,0.4360737204975733,0.4401700049222158,0.4401473105482878,0.4401212915991083,0.4400943990480715,0.4400694237215474,0.444095252384773,0.444067741566241,0.4440359772754752,0.4440029019018689,0.4439719673173388,0.448990974997294,0.4489584442123633,0.4489206007818745,0.4488808829620785,0.4488434563586114,0.4641372127842907,0.4640924073448026,0.4640392826533965,0.4639823989716804,0.4639277683211129,0.4776670574651534,0.4776132479141628,0.4775485079437879,0.4774781137667365,0.4774095149226167])

In [59]:
realVIX.shape

(30,)

In [64]:
initialSPX = np.array([0.09992093840981257,0.1781617982807032,0.2329235066714602,0.2749930226235151,0.3106782873547143,0.3422820828275765,0.4136692872971223,0.4799118705242865,0.06709033572913849,0.1030770804774587,0.1363788848334239,0.1667389423064116,0.1942112806390227,0.2191460100470349,0.2801283189375107,0.3313237149002883,0.04678646016073475,0.07706737451786527,0.1026386167084481,0.1257976614788078,0.1470885834874019,0.1669809914495011,0.2208814395382606,0.2660901298496009,0.00805724474535678,0.02619998314958867,0.04594411507825658,0.06353266005681651,0.0845012973891318,0.1029888528928549,0.1555681389458044,0.2010274887932842,0.00113129600711326,0.008799144330690551,0.02058895127311239,0.0343790256400592,0.04911691405801335,0.06425633203050402,0.1107907741822713,0.1531136597025941])

In [65]:
initialVIX = np.array([0.7497088142563657,0.7496925794246937,0.7496741366318193,0.7496552581010384,0.749637885755193,0.7544871438544895,0.7544644584834362,0.7544384498561026,0.754411567973601,0.7543866025551501,0.7590845992667851,0.759057078822633,0.7590253034180673,0.7589922164719897,0.7589612710640637,0.7646273075502549,0.7645947675448533,0.764556913388064,0.7645171843106657,0.7644797470989914,0.7807823437948596,0.7807375271756094,0.7806843892287504,0.7806274913537966,0.7805728470723017,0.7939488285269251,0.7938950067276593,0.7938302520211628,0.7937598418211995,0.7936912273630572])

In [60]:
optSPX = np.array([0.07478998330825991,0.1511881809624669,0.2027213742904438,0.2410147693292858,0.2731537060320192,0.3015950621576541,0.3634066339837972,0.4259388229055147,0.03423045697801266,0.05564390453620619,0.07821983386961903,0.09984314121849809,0.1197462200079833,0.1379057622601254,0.1810111849754902,0.2187701777850605,0.01335174762510088,0.0263698169715359,0.03824713629624694,0.0496782984407956,0.06059722951085358,0.07117409936709045,0.1020129208234652,0.1289857908825274,5.507653235168098e-06,0.0003417335484702847,0.001793874618338194,0.004031475693787042,0.008600844949581476,0.01362447892781697,0.03324074348211809,0.05432811081199039,-3.810543297612796e-08,2.314515445320785e-06,4.939699886146426e-05,0.0002662663642584691,0.0008058351466696318,0.001794208949200881,0.00842046207863016,0.01913447334959481])

In [61]:
optVIX = np.array([0.4361444591264983,0.4361281931076638,0.4361097148861902,0.4360908000898298,0.4360733943718457,0.4401696681720964,0.4401469737981951,0.4401209548490461,0.4400940622980407,0.440069086971546,0.4440949049733128,0.4440673941548383,0.4440356298641388,0.4440025544906016,0.4439716199061359,0.4489906132256151,0.4489580824407348,0.4489202390103048,0.4488805211905705,0.4488430945871615,0.4641367984359148,0.4640919929964292,0.464038868305026,0.4639819846233131,0.4639273539727485,0.4776665850977025,0.4776127755466941,0.477548035576298,0.4774776413992233,0.4774090425550807])

In [66]:
diffInitialSPX = np.abs(realSPX - initialSPX)

In [67]:
diffInitialVIX = np.abs(realVIX - initialVIX)

In [68]:
diffOptSPX = np.abs(realSPX - optSPX)
diffOptVIX = np.abs(realVIX - optVIX)

In [69]:
VIXstrike = np.array([0.07660333466800909, 0.07823036262505452, 0.08007866879151475,
                      0.08197064388101638, 0.08371167160297435, 0.07528231789011931,
                      0.07755350918528185, 0.08015741494276396, 0.08284874840358378,
                      0.0853482112450969,  0.07430112345712658, 0.077055734551228,
                      0.08023623852982013, 0.08354801898792094, 0.08664544590808211,
                      0.0732872469748735,  0.07654634796281677, 0.08033769704753065,
                      0.08431683207193616, 0.08806642128491488, 0.07084911309063555,
                      0.07534553012964565, 0.08067681958626018, 0.08638533974549638,
                      0.0918677586016848,  0.06910110193769552, 0.07450988995253548,
                      0.08101737363349071, 0.08809320258894406, 0.0949885695945464])

In [70]:
VIXexp = np.array([7, 7, 7, 7, 7, 14, 14, 14, 14, 14, 21, 21, 21, 21, 21, 30, 30, 30, 30, 30, 60, 60, 60, 60, 60, 90, 90, 90, 90, 90])

In [71]:
SPXstrike = np.array([0.9371, 0.8603, 0.8112, 0.7760, 0.7470, 0.7216, 0.6699,
                      0.6137, 0.9956, 0.9868, 0.9728, 0.9588, 0.9464, 0.9358,
                      0.9175, 0.9025, 1.0427, 1.0463, 1.0499, 1.0530, 1.0562,
                      1.0593, 1.0663, 1.0766, 1.2287, 1.2399, 1.2485, 1.2659,
                      1.2646, 1.2715, 1.2859, 1.3046, 1.3939, 1.4102, 1.4291,
                      1.4456, 1.4603, 1.4736, 1.5005, 1.5328])
SPXexp = np.array([30, 60, 90, 120, 150, 180, 270, 360,
                   30, 60, 90, 120, 150, 180, 270, 360,
                   30, 60, 90, 120, 150, 180, 270, 360,
                   30, 60, 90, 120, 150, 180, 270, 360,
                   30, 60, 90, 120, 150, 180, 270, 360])

In [77]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.dist=10
for i in range(30):
    ax.plot(np.array([VIXstrike[i], VIXstrike[i]]), np.array([VIXexp[i], VIXexp[i]]), np.array([diffInitialVIX.min()-0.0005, diffInitialVIX[i]]), 'y-.', alpha=0.5)
ax.scatter(VIXstrike, VIXexp, diffInitialVIX, s = 30, depthshade=False)
ax.set_xlabel('Strikes of VIX options', fontsize=24, labelpad=22)
ax.set_ylabel('Maturities of VIX options', fontsize=24, labelpad=22)
ax.set_zlabel('Errors', fontsize=24, labelpad=45)
plt.xticks(fontsize=20)
plt.yticks([7, 14, 21, 30, 60, 90], fontsize=20)
for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(20)
ax.tick_params(axis='z', which='major', pad=20)
plt.grid()

In [80]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.dist=10
ax.scatter(VIXstrike, VIXexp, diffOptVIX, s = 30, depthshade=False)
for i in range(30):
    ax.plot(np.array([VIXstrike[i], VIXstrike[i]]), np.array([VIXexp[i], VIXexp[i]]), np.array([diffOptVIX.min()-0.00000001, diffOptVIX[i]]), 'y-.', alpha=0.5)
ax.set_xlabel('Strikes of VIX options', fontsize=24, labelpad=22)
ax.set_ylabel('Maturities of VIX options', fontsize=24, labelpad=22)
ax.set_zlabel('Errors', fontsize=24, labelpad=20)
plt.xticks(fontsize=20)
plt.yticks([7, 14, 21, 30, 60, 90], fontsize=20)
for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(20)
ax.tick_params(axis='z', which='major', pad=5)
plt.grid()


In [85]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(SPXstrike, SPXexp, diffInitialSPX, s = 30, depthshade=False)
for i in range(40):
    ax.plot(np.array([SPXstrike[i], SPXstrike[i]]), np.array([SPXexp[i], SPXexp[i]]), np.array([diffInitialSPX.min()-0.01, diffInitialSPX[i]]), 'y-.', alpha=0.5)
ax.set_xlabel('Strikes of SPX options', fontsize=24, labelpad=22)
ax.set_ylabel('Maturities of SPX options', fontsize=24, labelpad=22)
ax.set_zlabel('Errors', fontsize=24, labelpad=40)
plt.xticks(fontsize=20)
plt.yticks([30, 60, 90, 120, 150, 180, 270, 360], fontsize=15)
for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(20)
ax.tick_params(axis='z', which='major', pad=20)
plt.grid()

In [84]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(SPXstrike, SPXexp, diffOptSPX, s = 30, depthshade=False)
for i in range(40):
    ax.plot(np.array([SPXstrike[i], SPXstrike[i]]), np.array([SPXexp[i], SPXexp[i]]), np.array([diffOptSPX.min()-0.00000001, diffOptSPX[i]]), 'y-.', alpha=0.5)
ax.set_xlabel('Strikes of SPX options', fontsize=24, labelpad=22)
ax.set_ylabel('Maturities of SPX options', fontsize=24, labelpad=22)
ax.set_zlabel('Errors', fontsize=24, labelpad=20)
plt.xticks(fontsize=20)
plt.yticks([30, 60, 90, 120, 150, 180, 270, 360], fontsize=15)
for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(20)
ax.tick_params(axis='z', which='major', pad=10)
plt.grid()